In [1]:
#dependencies and setup
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import plotly.express as px
from openpyxl import Workbook
import numpy as np
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine, text
from sqlalchemy import Column, Integer, String, Float
from pandas_profiling import ProfileReport

In [2]:
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine, text
from sqlalchemy import Column, Integer, String, Float
from pandas_profiling import ProfileReport
#  SQLite DB creation and establishing connection
database_path = "NJ_County_DB.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

In [3]:
sql_query = """
SELECT * FROM nj_zillow_house_value_index
 AS T1
INNER JOIN nj_population AS T2 ON T1.county_name = T2.county_name AND T1.year=T2.year
INNER JOIN nj_poverty_median_income AS T3 ON T1.county_name = T3.county_name AND T1.year=T3.year
INNER JOIN 
(SELECT county_name , year, AVG(tax_rate) AS tax_rate 
FROM nj_property_tax GROUP BY 1,2)AS T4 ON T1.county_name = T4.county_name AND T1.year=T4.year
INNER JOIN nj_mortgage_rates AS T5 ON T1.year=T5.year
;
"""
all_df = pd.read_sql(sql_query,sqlite_connection)
all_df.info()

2023-03-25 12:15:59,256 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
SELECT * FROM nj_zillow_house_value_index
 AS T1
INNER JOIN nj_population AS T2 ON T1.county_name = T2.county_name AND T1.year=T2.year
INNER JOIN nj_poverty_median_income AS T3 ON T1.county_name = T3.county_name AND T1.year=T3.year
INNER JOIN 
(SELECT county_name , year, AVG(tax_rate) AS tax_rate 
FROM nj_property_tax GROUP BY 1,2)AS T4 ON T1.county_name = T4.county_name AND T1.year=T4.year
INNER JOIN nj_mortgage_rates AS T5 ON T1.year=T5.year
;
")
2023-03-25 12:15:59,260 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-25 12:15:59,262 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
SELECT * FROM nj_zillow_house_value_index
 AS T1
INNER JOIN nj_population AS T2 ON T1.county_name = T2.county_name AND T1.year=T2.year
INNER JOIN nj_poverty_median_income AS T3 ON T1.county_name = T3.county_name AND T1.year=T3.year
INNER JOIN 
(SELECT county_name , year, AVG(tax_rate) AS tax_rate 
FROM nj_property_tax GR

In [4]:
all_df=all_df.loc[:,~all_df.columns.duplicated()].copy()
all_df

,county_name,year,num_of_bedrooms,house_value_index,est_pop,median_hh_income,poverty_count,poverty_rate,st_abb,state_code,county_code,tax_rate,apr_30,points_30,apr_15,points_15
0,ATLANTIC,2010,1,"120,414.14",274654,51457,36693,13.60,NJ,034,001,2.67,4.69,0.72,4.10,0.67
1,ATLANTIC,2011,1,"106,680.39",274635,49983,35108,13.10,NJ,034,001,2.65,4.45,0.72,3.68,0.71
2,ATLANTIC,2012,1,"100,139.16",274657,50881,38245,14.20,NJ,034,001,2.73,3.66,0.72,2.93,0.68
3,ATLANTIC,2013,1,"94,991.76",274360,51668,46281,17.10,NJ,034,001,2.87,3.98,0.73,3.10,0.70
4,ATLANTIC,2014,1,"92,839.52",272634,54208,40761,15.10,NJ,034,001,2.84,4.17,0.59,3.29,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,WARREN,2017,5,"337,688.95",105761,79633,7770,7.40,NJ,034,041,3.28,3.99,0.50,3.27,0.49
1256,WARREN,2018,5,"348,528.58",105709,77571,7006,6.70,NJ,034,041,3.32,4.54,0.48,4.00,0.45
1257,WARREN,2019,5,"352,652.50",105455,83998,7313,7.10,NJ,034,041,3.36,3.94,0.52,3.39,0.47
1258,WARREN,2020,5,"365,908.92",105624,80412,7539,7.30,NJ,034,041,3.40,3.11,0.73,2.60,0.69


In [5]:
all_df.to_csv('../Resources/final_data2.csv',index=False)

In [6]:
# close connection
sqlite_connection.close()